# First test to identify the impact of world events on music listening behaviour

Import the needed libraries

In [64]:
from datetime import datetime, timedelta
import scipy.stats
import codecs                     # load UTF-8 Content
import json                       # load JSON files
import pandas as pd               # Pandas handles dataframes
import numpy as np                # Numpy handles lots of basic maths operations
import matplotlib.pyplot as plt   # Matplotlib for plotting
import seaborn as sns             # Seaborn for beautiful plots
from dateutil import *            # I prefer dateutil for parsing dates
import math                       # transformations
import statsmodels.formula.api as smf  # for doing statistical regression
import statsmodels.api as sm      # access to the wider statsmodels library, including R datasets
from collections import Counter   # Counter is useful for grouping and counting
import scipy
from patsy import dmatrices
import statsmodels.formula.api as sm

Set global variables

N: the period, in days, before and after the event we are interested in

In [14]:
N=7

Set global functions

In [3]:
def read_events(a_file):
    events = pd.read_csv(a_file, sep=",", names=['id', 'description', 'year', 'month', 'day', 'category'])
    return events

# Event data

We load the events in a Pandas dataframe and create a new column that contains the date in one cell. The date is converted to datetime in order to make future processing easier. As we only have music data up till 2015, we filter out all the events after 01/01/2015.

In [4]:
events = read_events("data/events.csv")

dates = []
for index, row in events.iterrows():
    date = str(events.iloc[index]['year']) + "-" + str(events.iloc[index]['month']) + "-" + str(events.iloc[index]['day'])
    dates.append(date)

# CREATE A NEW DATA FRAME AND MERGE WITH EVENT DATA FRAME
df2 = pd.DataFrame(dates, columns=['date'])
events = pd.concat([events, df2], axis=1)
events['date'] = pd.to_datetime(events['date'], format='%Y-%m-%d')

# APPLY FILTER TO EVENT DATA FRAME
events_filtered = events[(events['year'] < 2015)]

print(events_filtered.head())

    id        description  year  month  day category       date
60  60           iPhone 6  2014      9    9     tech 2014-09-09
61  61  Samsung Galaxy S5  2014      2   25     tech 2014-02-25
62  62            Nexus 6  2014     10   16     tech 2014-10-16
63  63             Moto G  2014     11   28     tech 2014-11-28
64  64     Samsung Note 4  2014      8   31     tech 2014-08-31


# Load the listening event  data

We load the created data from the folder and load it into a dataframe. We convert the time for processing purposes.

In [5]:
df = pd.DataFrame(json.load(open("../data/allCountries_relativePlaycount_Genre.json")))
df['week'] = pd.to_datetime(df['week'], format='%Y-%m-%d')
print(df.head())

  country genre  playcount  relative_play       week  year
0      AT  punk         12       1.000000 2005-01-01  2005
1      AT  punk          4       0.173913 2005-02-19  2005
2      AT  punk         13       0.317073 2005-02-25  2005
3      AT  punk          1       0.016949 2005-03-04  2005
4      AT  punk          1       0.500000 2005-04-05  2005


Here, we can filter data in order to make processing faster. For now, we have concentrated ourselves at the US only.

In [6]:
# FILTER COUNTRIES
df_filter = df[(df.country == "US") & (df.genre == "total_playcount")]
print(df_filter.head())

#genres = list(set(df_filter['genre']))
years = list(set(df_filter['year']))

      country            genre  playcount  relative_play       week  year
49898      US  total_playcount        856            1.0 2005-01-01  2005
49899      US  total_playcount        508            1.0 2005-02-14  2005
49900      US  total_playcount        634            1.0 2005-02-15  2005
49901      US  total_playcount        635            1.0 2005-02-16  2005
49902      US  total_playcount        637            1.0 2005-02-17  2005


We perform a two-sample Welsh t-test, which assumes unequal variance. The first sample is the absolute playcount of songs N days prior to the event, the second sample comprises the absolute playcount 7 days after the event (including the event date itself. We have set the required significance level at .05.
We let the script print the event if there is a significant difference between the both. Furthermore, we create a plot for each significant event. 

From the plots, it looks like there is a significant seasonal effect that influences the data. However, this is not the case for all events.

In [7]:
for year in years:
    
    df_focus = df_filter[(df_filter.year == year)]
    
    if True:

        #df_filter_years_genres = df_filter_years[(df_filter.genre == genre)]

        for index, row in events_filtered.iterrows():
            dateEvent = row.date
            dateBefore = row.date - timedelta(days=N)
            dateAfter = row.date + timedelta(days=N)

            # ONLY FOR X DAYS BEFORE AND AFTER THE EVENT
            group_A = df_focus[(df_focus['week'] < dateEvent) & (df_focus['week'] > dateBefore)]
            group_B = df_focus[(df_focus['week'] >= dateEvent) & (df_focus['week'] < dateAfter)]
            
            if True:
                list_1 = list(group_A['playcount'])
                list_2 = list(group_B['playcount'])

                t_test = scipy.stats.ttest_ind(list_1, list_2, equal_var=False)

                # PRINT EVENT WHEN SIGNIFICANT
                if t_test[1] < 0.05:
                    print(row.description, row.date, t_test)
                    
                    plt.plot(group_A.week, group_A.playcount, color='r')
                    plt.plot(group_B.week, group_B.playcount)
                    plt.axvline(x=row.date, color="black", linestyle="--")
                    plt.xlim([dateBefore, dateAfter])
                    plt.xticks(rotation=45)
                    plt.xlabel('Date')
                    plt.ylabel('Absolute playcount')
                    plt.title(str(row.description) + '_' + str(row.year) + '-'+ str(row.month) + '-'+ str(row.day))
                    plt.savefig('../data/figures/' + str(row.description) + '_' + str(row.year) + '-'+ str(row.month) + '-'+ str(row.day) + '.png', dpi=300)
                    plt.close()

Baidu 2005-12-25 00:00:00 Ttest_indResult(statistic=3.1189328038721893, pvalue=0.010804563599950578)
iTunes 2005-12-25 00:00:00 Ttest_indResult(statistic=3.1189328038721893, pvalue=0.010804563599950578)
World of Warcraft 2005-03-25 00:00:00 Ttest_indResult(statistic=-2.2565977216037281, pvalue=0.03307451465256353)
Leonardo da Vinci 2005-04-15 00:00:00 Ttest_indResult(statistic=-3.3595002141328947, pvalue=0.0025342568965619713)
Facebook 2007-12-24 00:00:00 Ttest_indResult(statistic=5.1145946925120223, pvalue=0.00018552577304434541)
Dailymotion 2007-12-22 00:00:00 Ttest_indResult(statistic=6.1494645974682358, pvalue=8.2245887610422722e-06)
Webkinz 2007-12-25 00:00:00 Ttest_indResult(statistic=8.3070411901541021, pvalue=2.6580807466273438e-07)
YouTube 2007-12-23 00:00:00 Ttest_indResult(statistic=5.7894776439592341, pvalue=3.656173022689673e-05)
Second Life 2007-01-31 00:00:00 Ttest_indResult(statistic=-4.337358057998336, pvalue=0.00021705810835680352)
Hi5 2007-12-22 00:00:00 Ttest_indRes

/Users/eelcowiechert/anacona36/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3126: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/eelcowiechert/anacona36/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


BBB12 2012-01-17 00:00:00 Ttest_indResult(statistic=-3.7441968603115252, pvalue=0.00095944240883177466)
Samsung Galaxy S3 2012-12-25 00:00:00 Ttest_indResult(statistic=3.5890134562606559, pvalue=0.0023912843549986789)
Nexus 7 2012-12-25 00:00:00 Ttest_indResult(statistic=3.5890134562606559, pvalue=0.0023912843549986789)
Galaxy Note 2 2012-12-25 00:00:00 Ttest_indResult(statistic=3.5890134562606559, pvalue=0.0023912843549986789)
Play Station 2012-12-25 00:00:00 Ttest_indResult(statistic=3.5890134562606559, pvalue=0.0023912843549986789)
iPad 4 2012-12-25 00:00:00 Ttest_indResult(statistic=3.5890134562606559, pvalue=0.0023912843549986789)
Kindle Fire 2012-12-25 00:00:00 Ttest_indResult(statistic=3.5890134562606559, pvalue=0.0023912843549986789)
Kate Middleton Pictures Released 2012-12-13 00:00:00 Ttest_indResult(statistic=3.469862589790834, pvalue=0.0020103876066579146)
SOPA Debate 2012-01-20 00:00:00 Ttest_indResult(statistic=-2.7703129353285356, pvalue=0.010670834823118083)
Costa Concor

## Use data splitted into components

As we seems that the data is influenced by a seasonal component (re-occuring pattern), we create a new dataframe which includes a seperation of the original dataframe into a trend, seasonal and residual component.

In [119]:
df = pd.DataFrame(json.load(open("../data/TSA_one_genre_components.json")))
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['original'] = df['original'].astype('float64')
df['trend'] = df['trend'].astype('float64')
df['seasonal'] = df['seasonal'].astype('float64')
df['residual'] = df['residual'].astype('float64')

# CREATE DATA WITHOUT SEASONAL COMPONENT
df['no_seas'] = df['original'] - df['seasonal']

print(df.head())

  country       date            genre  original  residual   seasonal  trend  \
0      US 2005-01-01  total_playcount     856.0       NaN -34.666493    NaN   
1      US 2005-02-14  total_playcount     508.0       NaN  37.839847    NaN   
2      US 2005-02-15  total_playcount     634.0       NaN -19.994159    NaN   
3      US 2005-02-16  total_playcount     635.0       NaN -88.823265    NaN   
4      US 2005-02-17  total_playcount     637.0       NaN -11.010729    NaN   

      no_seas  
0  890.666493  
1  470.160153  
2  653.994159  
3  723.823265  
4  648.010729  


In [120]:
# FILTER COUNTRIES
df_filter = df[(df.country == "US")]

def get_year(year):
    return year.year

df_filter['year'] = df_filter['date'].map(get_year)

print(df_filter.head())
#genres = list(set(df_filter['genre']))
years = list(set(df_filter['year']))

  country       date            genre  original  residual   seasonal  trend  \
0      US 2005-01-01  total_playcount     856.0       NaN -34.666493    NaN   
1      US 2005-02-14  total_playcount     508.0       NaN  37.839847    NaN   
2      US 2005-02-15  total_playcount     634.0       NaN -19.994159    NaN   
3      US 2005-02-16  total_playcount     635.0       NaN -88.823265    NaN   
4      US 2005-02-17  total_playcount     637.0       NaN -11.010729    NaN   

      no_seas  year  
0  890.666493  2005  
1  470.160153  2005  
2  653.994159  2005  
3  723.823265  2005  
4  648.010729  2005  


/Users/eelcowiechert/anacona36/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Again, we perform a Welsh t-test

In [121]:
for year in years:
    
    df_focus = df_filter[(df_filter.year == year)]

    for index, row in events_filtered.iterrows():
        if row.year == year:
            dateEvent = row.date
            dateBefore = row.date - timedelta(days=N)
            dateAfter = row.date + timedelta(days=N)

            # ONLY FOR X DAYS BEFORE AND AFTER THE EVENT
            group_A = df_focus[(df_focus['date'] < dateEvent) & (df_focus['date'] > dateBefore)]
            group_B = df_focus[(df_focus['date'] >= dateEvent) & (df_focus['date'] < dateAfter)]
            
            list_1 = list(group_A['no_seas'])
            list_2 = list(group_B['no_seas'])

            t_test = scipy.stats.ttest_ind(list_1, list_2, equal_var=False)

            # PRINT EVENT WHEN SIGNIFICANT
            if t_test[1] < 0.05:
                print(row.description, row.date, t_test)
                
                plt.plot(group_A.date, group_A.no_seas, color='r')
                plt.plot(group_B.date, group_B.no_seas)
                plt.axvline(x=row.date, color="black", linestyle="--")
                plt.xlim([dateBefore, dateAfter])
                plt.ylim([0,max(max(list_1),max(list_2))+100])
                plt.xticks(rotation=45)
                plt.xlabel('Date')
                plt.ylabel('Absolute playcount')
                plt.title(str(row.description) + '_' + str(row.year) + '-'+ str(row.month) + '-'+ str(row.day))
                plt.savefig('../data/figures/No_seasonal_' + str(row.description) + '_' + str(row.year) + '-'+ str(row.month) + '-'+ str(row.day) + '.png', dpi=300)
                plt.close()

Ares 2005-10-15 00:00:00 Ttest_indResult(statistic=-3.0269343818929824, pvalue=0.011632296988400648)
Sky News 2005-07-07 00:00:00 Ttest_indResult(statistic=-2.918285250180316, pvalue=0.014400438356753505)
Facebook 2007-12-24 00:00:00 Ttest_indResult(statistic=3.0709156968927935, pvalue=0.011749488506499775)
Dailymotion 2007-12-22 00:00:00 Ttest_indResult(statistic=5.1985589262747061, pvalue=0.00036812998334475342)
Webkinz 2007-12-25 00:00:00 Ttest_indResult(statistic=4.7460580415376059, pvalue=0.0033934793524578568)
YouTube 2007-12-23 00:00:00 Ttest_indResult(statistic=3.390345338071711, pvalue=0.0068583240281245399)
eBuddy 2007-03-16 00:00:00 Ttest_indResult(statistic=-3.2149985833671182, pvalue=0.010972736601274369)
Hi5 2007-12-22 00:00:00 Ttest_indResult(statistic=5.1985589262747061, pvalue=0.00036812998334475342)
Club Penguin 2007-12-22 00:00:00 Ttest_indResult(statistic=5.1985589262747061, pvalue=0.00036812998334475342)
Tuenti 2008-12-25 00:00:00 Ttest_indResult(statistic=5.300483

/Users/eelcowiechert/anacona36/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3126: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/eelcowiechert/anacona36/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Samsung Note 4 2014-08-31 00:00:00 Ttest_indResult(statistic=3.4145364500301807, pvalue=0.040510625812618294)
ISIS 2014-08-20 00:00:00 Ttest_indResult(statistic=3.3449074643684353, pvalue=0.010073692955589559)
ALS Ice Bucket Challenge 2014-08-21 00:00:00 Ttest_indResult(statistic=5.3163423932748701, pvalue=0.00047786839181138206)
Conchita Wurst 2014-05-11 00:00:00 Ttest_indResult(statistic=4.3705885587622859, pvalue=0.0028023440161527783)
ISIS 2014-08-20 00:00:00 Ttest_indResult(statistic=3.3449074643684353, pvalue=0.010073692955589559)
Samsung Note 4 2014-08-31 00:00:00 Ttest_indResult(statistic=3.4145364500301807, pvalue=0.040510625812618294)
ISIS 2014-08-20 00:00:00 Ttest_indResult(statistic=3.3449074643684353, pvalue=0.010073692955589559)
ALS Ice Bucket Challenge 2014-08-21 00:00:00 Ttest_indResult(statistic=5.3163423932748701, pvalue=0.00047786839181138206)
Conchita Wurst 2014-05-11 00:00:00 Ttest_indResult(statistic=4.3705885587622859, pvalue=0.0028023440161527783)
ISIS 2014-08-2

# Perform the RDD

From the t-test, we can see that the placount already went down or up before the event, and that a found difference in the t-test is a logical finding. However, this can not be subscribed to the event. We are therfore looking to a discontiniuity in the data. Therefore we perform a RDD. 

The Forcing Variable (csize) is centered around the cutoff, and the cutoff predictor (small) is a dichotomous variable that indicates which side of the cutoff an observation lands.

In [67]:
for year in years:
    
    df_focus = df_filter[(df_filter.year == year)]
    
    if True:
        
        #df_filter_years_genres = df_filter_years[(df_filter.genre == genre)]

        for index, row in events_filtered.iterrows():
            if row.year == year:
                dateEvent = row.date
                dateBefore = row.date - timedelta(days=N)
                dateAfter = row.date + timedelta(days=N)
                
                # FUNCTION TO CLASSIFY THE DATS BEFORE THE EVENT WITH A 0, ON AND AFTER THE EVENT WITH A 1
                def small(size):
                    if(size>=dateEvent):
                        return 1
                    return 0
                
                def csize(size):
                    delta = size - dateEvent
                    return delta.days

                # ONLY FOR X DAYS BEFORE AND AFTER THE EVENT
                data = df_focus[(df_focus['week'] < dateAfter) & (df_focus['week'] >= dateBefore)]
                
                if not list(data['playcount']): # WHEN THE FILTERING RETURNS AN EMPTY DATAFRAME => CONTINUE
                    continue
                
                data['size'] = df_focus['week'].map(small)
                data['csize'] = df_focus['week'].map(csize)
                
                result = sm.ols(formula="playcount ~ size + csize", data=data).fit()
                
                # If sinificant, plot lines
                if result.pvalues['size'] <0.05:
                
                    plt.figure(num=None, figsize=(12, 6), dpi=80, facecolor='w', edgecolor='k')
                    plt.scatter(data.csize,data.playcount, color="blue")
                    l=data[data.csize<0].csize.count()
                    plt.plot(data.csize[0:l], result.predict()[0:l], '-', color="r")
                    plt.plot(data.csize[l:], result.predict()[l:], '-', color="r")
                    plt.axvline(x=-0.5,color="black", linestyle="--")
                    plt.xlabel('Days before and after the event')
                    plt.ylabel('Playcount songs')
                    plt.title("Regression Discontinuity: Reading playcount by date Before and After the Event \n" + str(row.description) + '_' + str(row.year) + '-'+ str(row.month) + '-'+ str(row.day)), fontsize="18")
                    plt.savefig('../data/figures/RDD_' + str(row.description) + '_' + str(row.year) + '-'+ str(row.month) + '-'+ str(row.day) + '.png', dpi=300)


/Users/eelcowiechert/anacona36/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/eelcowiechert/anacona36/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/eelcowiechert/anacona36/anaconda/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may co